## 数据格式
jmdata是个多层词典数据。

jmdata.keys() = ['judge', 'article']

jmdata['judge'].keys() 和 jmdata['article'].keys()为文章的id

jmdata['judge']['JiemoduiArticleItem/1165'].keys() = ['id', 'appid', 'item_id', 'user_id', 'judge_json', 'time_stamp', 'done_tag', 'tid', 'insert_time', 'last_commit_time_stamp', 'worthiness', 'has_comment'], 其中最主要数据为'judge_json'

jmdata['judge']['JiemoduiArticleItem/1165']['judge_json']为一个字典列表，列出了 词句标注 段落标注 和 实体标注

In [617]:
def getLabel(val):
    """
    根据val词典获取label
    """
    if 'sub' in val.keys():
        val = val['sub']
    else:
        global noLabelCount
        noLabelCount += 1
        return ''
    s = ''
    while True:
        s += (val['name'] + '|')
        if 'sub' in val.keys():
            val = val['sub']
        else:
            break
    return s.strip('|')

def hasCN(s):
    """
    判断一段文本是否有中文
    """
    zhPattern = re.compile(u'[\u4e00-\u9fa5]+')
    match = zhPattern.search(s)
    return match != None


## 任务：
统计文章中有多少比例的句子是从PDF中摘取的

## 流程
PDF句子与表格抽取 --> 生成pdf句子列表

文章句子抽取 --> 生成文章句子列表

for 句子 in 文章句子列表：

    if 句子与pdf中某句子匹配：
    
        count+=1
        
比例：count/len(文章句子列表) * 100%

## 问题：
1.有些文章包含了多家公司，而pdf只是一家公司的财报

2.文章中的句子不一定是全面摘抄的，可能做了修改，所以需要评价句子之间的相似度

In [21]:
jmdata_judge['JiemoduiArticleItem/62681']

{'_tid': 'Quality-2017-05-19 10:13:27.320667-266',
 'appid': 'earnings_reporter',
 'done_tag': 1,
 'has_comment': 0,
 'id': 11800,
 'insert_time': '2017-05-19 10:53:27.387305',
 'item_id': 'JiemoduiArticleItem/62681',
 'judge_json': [{'detail': {'belong_to': [1],
    'mark_id': 'AB8EED24201CF73D201EAE3DF348EF81',
    'pos': {'length': 58, 'start': 214},
    'text': ['上月，秀强股份又公布了拟 3.8 亿元收购培基教育的消息，后者是面向1.5--12 岁儿童提供学前教育与培训的机构。'],
    'val': {'name': '意图',
     'sub': {'name': '企业活动', 'sub': {'name': '投资活动', 'sub': {'name': '收购'}}}}},
   'timestamp': 0,
   'type': 'text'},
  {'alias': '段落',
   'detail': {'belong_to': [3],
    'mark_id': 'FC7E741CAFFE6897B7141B4BC66AD53E',
    'text': ['秀强玻璃工艺股份有限公司创立于 1992 年，是中国最大的家电玻璃制造商之一。 2015 年 12 月，秀强股份发布公告称将以现金 2.1 亿元购买杭州全人教育集团有限公司 100% 股权，将主营业务向玻璃与教育双轮驱动的双主业模式转变。 此后，秀强股份参股设立的新余修齐平治基金 1.5 亿元收购教育信息化公司领信教育 36% 的股权。2016 年 5 月，秀强股份公开信息称拟收购幼教投资运营机构徐幼集团 51% 以上股权。上月，秀强股份又公布了拟 3.8 亿元收购培基教育的消息，后者是面向1.5--12 岁儿童提供学前教育与培训的机构。'],
    'val': {'name': '意图',
     '

In [14]:
dataJoin

{'JiemoduiArticleItem/46944': 'http://disclosure.szse.cn/finalpage/2016-04-26/1202239317.PDF',
 'JiemoduiArticleItem/46971': 'http://disclosure.szse.cn/finalpage/2016-04-26/1202239571.PDF',
 'JiemoduiArticleItem/47091': 'http://disclosure.szse.cn/finalpage/2016-04-30/1202283035.PDF',
 'JiemoduiArticleItem/61452': 'http://disclosure.szse.cn/finalpage/2016-08-26/1202622898.PDF',
 'JiemoduiArticleItem/62486': 'http://disclosure.szse.cn/finalpage/2016-10-27/1202791188.PDF',
 'JiemoduiArticleItem/62535': 'http://disclosure.szse.cn/finalpage/2016-10-18/1202765393.PDF',
 'JiemoduiArticleItem/62536': 'http://disclosure.szse.cn/finalpage/2016-10-18/1202763826.PDF',
 'JiemoduiArticleItem/62661': 'http://disclosure.szse.cn/finalpage/2016-10-25/1202780794.PDF',
 'JiemoduiArticleItem/62681': 'http://disclosure.szse.cn/finalpage/2016-10-25/1202780960.PDF',
 'JiemoduiArticleItem/62692': 'http://disclosure.szse.cn/finalpage/2016-10-26/1202785562.PDF',
 'JiemoduiArticleItem/62745': 'http://disclosure.s

In [13]:
#mapping数据与jmdata的交集文章
#dataJoin = {pdfurl: jmdId}
#使用jmdurl去匹配jmdata_mapping和jmdata_article
dataJoin = {}
for data_map in jmdata_mapping[2:]:
    for key in jmdata_article.keys():
        if jmdata_article[key]['url'] == data_map['jmd_url']:
            dataJoin[key] = data_map['pdf_url']
#         if jmdata_article[key]['url'] == 'http://disclosure.szse.cn/finalpage/2016-10-27/1202791188.PDF':
#             print(key)

In [1024]:
l = ['http://disclosure.szse.cn/finalpage/2017-04-26/1203386046.PDF',
       'http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2017-04-29/600661_2017_1.pdf',
       'http://disclosure.szse.cn/finalpage/2016-10-18/1202765393.PDF',
       'http://disclosure.szse.cn/finalpage/2016-10-18/1202763826.PDF',
       'http://disclosure.szse.cn/finalpage/2016-10-25/1202780794.PDF',
       'http://disclosure.szse.cn/finalpage/2016-10-27/1202793355.PDF',
       'http://disclosure.szse.cn/finalpage/2016-10-25/1202780960.PDF',
       'http://disclosure.szse.cn/finalpage/2016-10-27/1202791188.PDF',
       'http://disclosure.szse.cn/finalpage/2016-10-26/1202785562.PDF',
       'http://disclosure.szse.cn/finalpage/2017-04-24/1203360123.PDF',
       'http://www.cninfo.com.cn/finalpage/2017-04-27/1203451818.PDF']
for d in dataJoin.keys():
    if not dataJoin[d] in l:
        print(d)

JiemoduiArticleItem/68186
JiemoduiArticleItem/46971
JiemoduiArticleItem/68239
JiemoduiArticleItem/68838
JiemoduiArticleItem/71019
JiemoduiArticleItem/68695
JiemoduiArticleItem/70280
JiemoduiArticleItem/70486
JiemoduiArticleItem/70887
JiemoduiArticleItem/47091
JiemoduiArticleItem/61452
JiemoduiArticleItem/68751
JiemoduiArticleItem/46944


In [1]:
import pickle
fp = open('/home/zhuyuhe/mydata/jmd/jmt_jmd_mappings.pkl', 'rb')
jmdata_mapping = pickle.load(fp, encoding = 'utf')
with open('/home/zhuyuhe/mydata/jmd/jmt_jmd_mappings2.pkl', 'wb') as pickle_file:
    pickle.dump(jmdata_mapping, pickle_file , protocol=2)

In [1]:
import pickle
fp = open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_mapping.pkl', 'rb')
jmdata_mapping = pickle.load(fp, encoding = 'utf')
# with open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_mapping2.pkl', 'wb') as pickle_file:
#     pickle.dump(jmdata_mapping, pickle_file , protocol=2)

In [5]:
k = jmdata_mapping[0]
print(k)

{'jmd_url': 'https://www.jiemodui.com/N/47007', 'year': '2016', 'pdf_url': 'http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2016-04-27/600661_2016_1.pdf', 'content': '\n芥末堆讯，近日，新南洋、立思辰、保龄宝、珠江钢琴、世纪鼎利、秀强股份6家A股公司相继发布2016年一季度财报。 新南洋2016年一季度：业绩增长65%超预期，关注K12与职教长期布局\n \n财报显示，新南洋2016年一季度实现营收2.89亿元，同比增长19.62%；归母净利润1384.43万，同比增长65.08%；EPS为0.05元。收入与业绩增长的主要原因是教育培训与服务板块稳健增长，而原有精密制造业务和数字电视与信息服务业务调整与处置已与上年逐步完成，负面影响逐渐降低。公司预计2016年实现营业收入12.5亿元，一季度完成全年目标23.12%。 \n定增募集资金主要用于拓展K12与职业教育两大板块，分别组建上海南洋昂立教育发展有限公司和上海新南洋职业教育发展有限公司。K12预期将新设60个学习中心重点拓展上海及长三角地区的K12教育业务，职教将在3年内预计与不少于10所高职院校合作，帮助其实现专业设置、课程体系设计、实训服务等方面的改善，提升其办学水平和学生就业，即以轻资产教研及管理服务输出方式实现公司长期收入与业绩的提升。 立思辰2016年一季度：营收1.86亿、归母净利润1227.04万元\n \n财报显示，立思辰2016年一季度实现营收1.86亿元，同比增长54.64%；归母净利润1227.04万元，同比扭亏为盈。收入增长主要因为康邦科技及江南信安3月份并表，以及敏特昭阳2015年7月纳入合并上年同期对比数据未包含。 \n报告期内公司中标北京市通州区教育云平台一期建设项目、成都市教育技术装备管理中心\n4/7/9中学办学条件提升项目、北京市朝阳区数字校园云平台维护项目、宁夏部分县市教育信息化建设项\n目，并积极参与部分学校的教育信息化建设。截至目前，在手的教育业务订单金额近五亿元。 保龄宝2016年一季度：营收2.75亿、归母净利润1332.42万元\n \n财报显示，保龄宝2016年一季度实现

In [962]:
def MatchTwoText(t1, t2, threshold = 0.5):
    """
    检查两个句子是否匹配
    """
    ct1 = ' '.join(jieba.cut(t1)).split()
    ct2 = ' '.join(jieba.cut(t2)).split()
    commonL = set(ct1).intersection(set(ct2)).__len__()
    if commonL / set(ct1).__len__() > threshold:
        print("match " + str(commonL / set(ct1).__len__() * 100) + '% ')
        print("文章句子:" + '\n' + t1)
        print("PDF句子: " + '\n' + t2)
        print('\n')
        return True
    return False
    
def isMatch(text, pdfTextList, threshold = 0.5):
    """
    @parameter text: 待检查句子
    @parameter pdfTextList: PDF句子与表格信息列表
    @parameter threshold: 相似度为多少时返回true
    @return： boolean值，pdf列表中是否有与待检查句子非常相似的句子
    """
    for pdftext in pdfTextList:
        if MatchTwoText(text, pdftext, threshold):
            return True
    return False

a = "创显将大力发展软硬件结合的教学解决方案，配套以不同层次的专业培训服务"
pdflist = ['2017年，公司将大力发展软硬件结合的视频教学解决方案，形成精品录播室，标准录播，配套以不同的专业培训服务', '2017年，公司将在既有的教师化信息能力平台业务基础上，进一步深化细化培养服务']
print(isMatch(a, pdflist, 0.6))


match 86.66666666666667% 
文章句子:
创显将大力发展软硬件结合的教学解决方案，配套以不同层次的专业培训服务
PDF句子: 
2017年，公司将大力发展软硬件结合的视频教学解决方案，形成精品录播室，标准录播，配套以不同的专业培训服务


True


In [946]:
fileObject = open('/home/zhuyuhe/mydata/jmd/percent/cxkjTextList.txt')
textlist = fileObject.read().split('\n')
fileObject = open('/home/zhuyuhe/mydata/jmd/percent/cxkjTableList.txt')
tablelist = fileObject.read().split('\n')
fileObject.close()

PDFTextList = []
PDFTextList.append(textlist)
PDFTextList.append(tablelist)
PDFTextList = sum(PDFTextList, [])
PDFTextList.__len__()

608

In [951]:
count = 0
for text in cxkj:
    if isMatch(text, PDFTextList, 0.7):
        count+=1
print(str(count / cxkj.__len__() * 100) + '% 的句子来自PDF')

match 77.77777777777779% 
文章句子:
净利润 5156.24 万元，比 2015 年度增长 73.82%
PDF句子: 
净利润 51,562,442.34 元，比 2015 年度增长 73.82%


match 89.65517241379311% 
文章句子:
 主营业务 创显科教智慧教育综合解决方案提供商和服务运营商，立足于中小学基础教育领域，同时涉及学前教育和职业教育，围绕国家教育部“三通两平台”建设开展业务
PDF句子: 
公司是智慧教育综合解决方案提供商和服务运营商，立足于中小学基础教育领域，同时涉及学前教育和职业教育，围绕国家教育部“三通两平台”建设开展业务


match 95.74468085106383% 
文章句子:
 创显科教主要面向教育管理机构、学校，销售由智慧教育云平台和智能教学终端、智慧教育增值服务融合形成的“班班通”、“校校通”、“人人通”、“教研培训”解决方案，当前已经具有大屏显示和交互式触控的智能硬件核心技术、智慧教育云平台、基于大数据的智慧学习等软件核心技术
PDF句子: 
公司主要面向教育管理机构、学校，销售由智慧教育云平台和智能教学终端、智慧教育增值服务融合形成的“班班通”、“校校通”、“人人通”、 “教研培训”解决方案，当前已经具有大屏显示和交互式触控的智能硬件核心技术、智慧教育云平台、 基于大数据的智慧学习等软件核心技术


match 100.0% 
文章句子:
 S 模式（Solution 方案销售模式），是直接面向用户提供解决方案，主要通过公司销售团队以品牌直营方式、服务化运营模式来进行销售
PDF句子: 
S  模式（Solution  方案销售模式），是直接面向用户提供专业化和系统性的解决方案，主要通过公司专业化的销售团队以品牌直营方式、服务化运营模式来进行销售


match 71.42857142857143% 
文章句子:
这是公司重点发展的销售模式
PDF句子: 
报告期内，公司实行“PSG”立体销售模式，充分发挥公司“硬件+软件+服务+内容”的综合竞争优势，发挥公司“政产学研用”的深度服务型营销推广优势，发挥产业联盟的资源共享优势；有助于公司在保证常规发展的基础上，实现更快的发展，赢得市场的主控权


match 100.0% 
文章句子:
 G 模式（GeneralCon

In [948]:
fp = open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_mapping.pkl', 'rb')
jmdata_mapping = pickle.load(fp, encoding = 'utf')
jmdata_mapping = jmdata_mapping[2:]

cxkj = jmdata_mapping[13]['content']
cxkj = re.split('[。；]', cxkj.strip())
cxkj = [x.replace('\n','') for x in cxkj]

In [7]:
f = open('/home/zhuyuhe/mydata/jmd/jmd_judge_1220.pkl', 'rb')
f_article = open('/home/zhuyuhe/mydata/jmd/jmd_judge_1220_updated.pkl', 'rb')
jmdata_judge = pickle.load(f, encoding='utf')['judge']
jmdata_article = pickle.load(f_article, encoding = 'utf')['article']

In [9]:
#数据探索
types = []
textEntites = [] #实体
textText = []    #词句
textParagraph = [] #段落
textTextComment = [] #词句批注
textImage = [] #图片标签
count = 0

#itemKeys 为文章id集合
# itemKeys = jmdata_judge.keys()           根据需要选择item集合！！！！！！！！！！
itemKeys = dataJoin.keys()
f = open('/home/zhuyuhe/mydata/jmd/percent/dataJoin.txt', 'w')
for itemkey in itemKeys:
    itemJudgeJson = jmdata_judge[itemkey]['judge_json']
    count += len(itemJudgeJson)
    for jjk in itemJudgeJson:
        types.append(jjk['type'])
        if (jjk['detail']['val']['name'] == '实体'):
            textEntites.append(jjk)
        if jjk['type'] == 'text-comment':
            textTextComment.append(jjk)
        if (jjk['type'] == 'image-tag') | (jjk['type'] == 'image-tag-comment'):
            textImage.append(jjk)
        if (jjk['type'] == 'text') & (jjk['detail']['val']['name'] != '实体'):
            textText.append(jjk)
            f.write(','.join(jjk['detail']['text']).replace('\n', '') + '\t' + getLabel(jjk['detail']['val']) + '\t' + dataJoin[itemkey])
            f.write('\n')
        if (jjk['type'] == 'paragraph') | (jjk['type'] == 'paragraph-comment'):
            textParagraph.append(jjk)
#             f.write(','.join(jjk['detail']['text']).replace('\n', '') + '\t' + getLabel(jjk['detail']['val']) + '\t' + dataJoin[itemkey])
#             f.write('\n')
print("数据信息如下: ")
print("一共有 " + str((len(itemKeys))) + " 篇文章， 文章id为'judge'的key")
print("一共有 " + str(count) + " 条标注")
print("实体 标注一共有 " + str(len(textEntites)) + "条")
print("词句批注 标注一共有 " + str(len(textTextComment)) + "条")
print("图片标签 标注一共有 " + str(len(textImage)) + " 条， 没有文本，是对图片内容的标注，可以去除")
print("词句 标注一共有 " + str(len(textText)) + " 条")
print("段落 标注一共有 " + str(len(textParagraph)) + "条")
f.close()

数据信息如下: 
一共有 24 篇文章， 文章id为'judge'的key
一共有 1048 条标注
实体 标注一共有 760条
词句批注 标注一共有 5条
图片标签 标注一共有 18 条， 没有文本，是对图片内容的标注，可以去除
词句 标注一共有 178 条
段落 标注一共有 87条


In [1005]:
dataJoin

{'JiemoduiArticleItem/46944': 'http://disclosure.szse.cn/finalpage/2016-04-26/1202239317.PDF',
 'JiemoduiArticleItem/46971': 'http://disclosure.szse.cn/finalpage/2016-04-26/1202239571.PDF',
 'JiemoduiArticleItem/47091': 'http://disclosure.szse.cn/finalpage/2016-04-30/1202283035.PDF',
 'JiemoduiArticleItem/61452': 'http://disclosure.szse.cn/finalpage/2016-08-26/1202622898.PDF',
 'JiemoduiArticleItem/62486': 'http://disclosure.szse.cn/finalpage/2016-10-27/1202791188.PDF',
 'JiemoduiArticleItem/62535': 'http://disclosure.szse.cn/finalpage/2016-10-18/1202765393.PDF',
 'JiemoduiArticleItem/62536': 'http://disclosure.szse.cn/finalpage/2016-10-18/1202763826.PDF',
 'JiemoduiArticleItem/62661': 'http://disclosure.szse.cn/finalpage/2016-10-25/1202780794.PDF',
 'JiemoduiArticleItem/62681': 'http://disclosure.szse.cn/finalpage/2016-10-25/1202780960.PDF',
 'JiemoduiArticleItem/62692': 'http://disclosure.szse.cn/finalpage/2016-10-26/1202785562.PDF',
 'JiemoduiArticleItem/62745': 'http://disclosure.s

In [987]:
#摘取dataJoin的段落内容
#将文本及其标签映射到词典当中
dataDic = {}
data = [textParagraph, textText]
noLabelCount = 0
for d in data:
    for detail in d:
        dataDic[','.join(detail['detail']['text'])] = getLabel(detail['detail']['val'])

#从词典构建dataframe
s = pd.Series(dataDic)
s.index_name = 'name'
jmdata = s.reset_index()
jmdata.columns = ['文本', '意图']

ValueError: Length of values does not match length of index

In [1044]:
cxkjPDF = pd.read_csv('/home/zhuyuhe/mydata/jmd/predict/pdftext.txt', header=None, sep='\t')
cxkjPDF.columns = ['文本']
replaceEntityDict = {'时间': '时间', '媒体': '媒体', '企业': '企业', '人名': '人名', '职务/身份': '身份', '简称/缩写（形式）': '简称', '业务/服务': '业务', '产品': '产品', '行业/领域/市场': '领域', '组织/团体/机构/基金': '组织', '股票': '股票', '国家/地区': '地点', '政策/规划': '政策', '资质/资格/证书': '资格', '协议/合同': '协议', '借代': '借代'}
for entity in replaceEntityDict.keys():
    entitylist = entitesDic[entity]
    for en in entitylist:
        cxkjPDF.文本 = cxkjPDF.文本.apply(lambda x: x.replace(en, replaceEntityDict[entity]))
stop_words = pd.read_table('/home/zhuyuhe/mydata/jmd/stopwords.txt', header = None)
stop_words = stop_words[0].tolist()
for word in stop_words:
    cxkjPDF.文本 = cxkjPDF.文本.apply(lambda x: x.replace(word, ''))

#去除标点符号
punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒
﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠
々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻
︽︿﹁﹃\t﹙+﹛﹝（｛“‘-—_…''')
# 对str/unicode
cxkjPDF.文本 = cxkjPDF.文本.apply(lambda s: ''.join(filter(lambda x: x not in punct, s)))

        
#载入自定义词典和自定义停止词
jieba.load_userdict('/home/zhuyuhe/mydata/jmd/entites.txt')
jieba.analyse.set_stop_words('/home/zhuyuhe/mydata/jmd/stopwords.txt')
def splitCN(x):
    return ' '.join(jieba.cut(x))
    #return ''.join(jieba.analyse.extraxt_tags(x))
cxkjPDF.文本 = cxkjPDF.文本.apply(lambda x: splitCN(x))

def digit2CN(x):
    wordlist = x.split()
    result = x
    for word in wordlist:
        if word.isdigit():
            result = result.replace(word, '数字')
        if isPercent(word):
            result = result.replace(word, '百分比')
        if not hasCN(word):
            result = result.replace(word, '')
    return result
cxkjPDF.文本 = cxkjPDF.文本.apply(lambda x: digit2CN(x))
cxkjPDF.to_csv('/home/zhuyuhe/mydata/jmd/predict/cxkjpdftext_processed.txt', header = None, index = None)

In [693]:
#将文本及其标签映射到词典当中
dataDic = {}
data = [textParagraph, textText]
noLabelCount = 0
for d in data:
    for detail in d:
        dataDic[','.join(detail['detail']['text'])] = getLabel(detail['detail']['val'])

#从词典构建dataframe
s = pd.Series(dataDic)
s.index_name = 'name'
jmdata = s.reset_index()
jmdata.columns = ['文本', '意图']
#只分类二级类目
jmdata.意图 = jmdata.意图.apply(lambda x: '|'.join(x.split('|')[:2]))

labelList = jmdata.意图.value_counts()[jmdata.意图.value_counts().values > 50].index.tolist()
labelList.remove('引导性文字')
#去除样本数目太少的类别样本
jmdata = jmdata[jmdata.意图.apply(lambda x: x in labelList)]
#去除不包含文字的文本（有些图片标签标注错误）
jmdata = jmdata[jmdata.文本.apply(lambda x: hasCN(x))]

entitesDic = {}
for entity in textEntites:
    key = getLabel(entity['detail']['val'])
    if not key in entitesDic.keys():
        entitesDic[key] = []
    if 'text' in entity['detail'].keys():
        entitesDic[key].append(' '.join(entity['detail']['text']))

for ent in entitesDic.keys():
    entitesDic[ent] = list(set(entitesDic[ent]))
    
#从替代表中去除我们不想替代的重要词汇    
entitesDic['国家/地区'].remove('美')
entitesDic['时间'].remove('同期')
entitesDic['时间'].remove('好未来')
entitesDic['组织/团体/机构/基金'].remove('企业')
entitesDic['时间'].remove('增长')
entitesDic['时间'].remove('同比增长')


replaceEntityDict = {'时间': '时间', '媒体': '媒体', '企业': '企业', '人名': '人名', '职务/身份': '身份', '简称/缩写（形式）': '简称', '业务/服务': '业务', '产品': '产品', '行业/领域/市场': '领域', '组织/团体/机构/基金': '组织', '股票': '股票', '国家/地区': '地点', '政策/规划': '政策', '资质/资格/证书': '资格', '协议/合同': '协议', '借代': '借代'}
for entity in replaceEntityDict.keys():
    entitylist = entitesDic[entity]
    for en in entitylist:
        jmdata.文本 = jmdata.文本.apply(lambda x: x.replace(en, replaceEntityDict[entity]))

#自定义停止词集合，并将停止词从文本中去除
stop_words = pd.read_table('/home/zhuyuhe/mydata/jmd/stopwords.txt', header = None)
stop_words = stop_words[0].tolist()
for word in stop_words:
    jmdata.文本 = jmdata.文本.apply(lambda x: x.replace(word, ''))

#去除标点符号
punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒
﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠
々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻
︽︿﹁﹃\t﹙+﹛﹝（｛“‘-—_…''')
# 对str/unicode
jmdata.文本 = jmdata.文本.apply(lambda s: ''.join(filter(lambda x: x not in punct, s)))

        
#载入自定义词典和自定义停止词
jieba.load_userdict('/home/zhuyuhe/mydata/jmd/entites.txt')
jieba.analyse.set_stop_words('/home/zhuyuhe/mydata/jmd/stopwords.txt')
def splitCN(x):
    return ' '.join(jieba.cut(x))
    #return ''.join(jieba.analyse.extraxt_tags(x))
jmdata.文本 = jmdata.文本.apply(lambda x: splitCN(x))

def digit2CN(x):
    wordlist = x.split()
    result = x
    for word in wordlist:
        if word.isdigit():
            result = result.replace(word, '数字')
        if isPercent(word):
            result = result.replace(word, '百分比')
        if not hasCN(word):
            result = result.replace(word, '')
    return result
jmdata.文本 = jmdata.文本.apply(lambda x: digit2CN(x))

jmdata.意图 = jmdata.意图.apply(lambda x: '__label__' + x)

jmdata_train = pd.DataFrame()
jmdata_test = pd.DataFrame()
labelset = list(set(jmdata.意图))
for l in labelset:
    label_item = jmdata[jmdata.意图 == l]
    jmdata_train = pd.concat([label_item.head(int(len(label_item)*0.7)), jmdata_train])
    jmdata_test = pd.concat([label_item.tail(int(len(label_item)*0.3)), jmdata_test])
    
#打乱数据集    
jmdata_train = shuffle(jmdata_train)
jmdata_test = shuffle(jmdata_test)

jmdata_train.to_csv('/home/zhuyuhe/mydata/jmd/jmdata_cut_train.txt', sep = ' ', index = False, quoting=csv.QUOTE_NONE, quotechar='',escapechar= ' ', header=False)
jmdata_test.to_csv('/home/zhuyuhe/mydata/jmd/jmdata_cut_test.txt', sep = ' ', index = False, quoting=csv.QUOTE_NONE, quotechar='',escapechar= ' ', header=False)

jmdata.head(20)

,文本,意图
0,归属于 企业 净利润 为 数字 万美元 时间 为 数字 万美元 降幅 百分比 其中 汇兑损失...,__label__企业业绩|现状
1,归属于 企业 净利润 从 时间 数字 万美元 增长 到 时间 财年 数字 万美元 增幅 百分比,__label__企业业绩|现状
2,截至 时间 年 时间 数字 日 企业 共有 数字9 个 教学 简称 截至 时间 年 时间 数...,__label__企业活动|经营活动（业务）
3,时间 国 网络 业务 市场 规模 约 数字 亿元 时间 年 已 达到 数字 亿元 实现 百分...,__label__企业业绩|前景（预测）
5,时间 年 数字 月 企业 股份 发布公告 称 将 以 现金 数字 亿...,__label__企业活动|投资活动
6,时间 企业 承接 企业 业务 以及 企业 业务,__label__企业活动|经营活动（业务）
8,时间 企业 业务 平台 产品 企业 同学 正式 上线,__label__企业活动|经营活动（业务）
9,因此 企业 将 积极 投入 业务 及 业务 网络 领域 开发 平台 开发 加强 基础 业...,__label__企业活动|经营活动（业务）
10,归属于 企业 非 企业 业务 准则 净利润 不 考虑 股权激励费用 从 时间 数字 万美...,__label__企业业绩|现状
11,身份 综合 业务 入学人数 较 时间 下降 百分比,__label__企业业绩|现状


In [445]:
jmdata_train.to_csv('/home/zhuyuhe/mydata/jmd/jmdata_cut_train.txt', sep = ' ', index = False, quoting=csv.QUOTE_NONE, quotechar='',escapechar= ' ', header=False)
jmdata_test.to_csv('/home/zhuyuhe/mydata/jmd/jmdata_cut_test.txt', sep = ' ', index = False, quoting=csv.QUOTE_NONE, quotechar='',escapechar= ' ', header=False)

In [22]:
jmdata.意图.value_counts()

NameError: name 'jmdata' is not defined

## 使用svm模型

In [713]:
import math
docList = jmdata.文本.tolist()
all_dict = {}
#计算idf值
idfDic = {}
total = 0
for line in docList:
    temp_dict = {}
    total += 1
    for word in line.split():
        temp_dict[word] = 1
    for key in temp_dict:
        num = all_dict.get(key, 0)
        all_dict[key] = num + 1
for key in all_dict:
    p = '%.10f' % (math.log10(total/(all_dict[key] + 1)))
    idfDic[key] = p

#将idf词典写入txt文件
f = open('/home/zhuyuhe/mydata/jmd/idf.txt', 'w')
for key, value in idfDic.items():
    f.write(key + ' ' + str(value))
    f.write('\n')
f.close()

#根据tfidf抽取关键词,并记录关键词的tfidf值
jieba.analyse.set_idf_path('/home/zhuyuhe/mydata/jmd/idf.txt')
jmdata['keywords'] = ''
jmdata['tfidf'] = ''
for i in jmdata.index:
    jmdata.loc[i, 'keywords'] = jieba.analyse.extract_tags(jmdata.loc[i, '文本'], topK = 15)
    jmdata.loc[i, 'tfidf'] = jieba.analyse.extract_tags(jmdata.loc[i, '文本'], topK = 15, withWeight=True)

#获取词袋
#格式：{词语：编号}
bow = list(set(jmdata.keywords.apply(lambda x: ' '.join(x) + ' ').sum().split()))
#为各个词语分配编号
#now - number of words
now = {}
count = 1
for w in bow:
    now[w] = count
    count += 1

#建立label类别数字映射
labelDict = {}
labes = jmdata.意图.unique()
n = 0
for l in labels:
    labelDict[l] = n
    n += 1
jmdata['label'] = jmdata.意图.apply(lambda x: labelDict[x])

#建立词袋数据，特征值为此词语在此文本中的tfidf值
#这一步速度很慢！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！
for i in range(len(bow)):
    jmdata[i] = 0

for i in jmdata.index:
    for key, value in dict(jmdata.loc[i, 'tfidf']).items():
        jmdata.loc[i, now[key]] = value

#划分训练集与测试集
X_train = pd.DataFrame()
X_test = pd.DataFrame()
labelset = list(set(jmdata.意图))
for l in labelset:
    label_item = jmdata[jmdata.意图 == l]
    X_train = pd.concat([label_item.head(int(len(label_item)*0.7)), X_train])
    X_test = pd.concat([label_item.tail(int(len(label_item)*0.3)), X_test])
    

y_train = X_train.label
y_test = X_test.label

In [754]:
clf = OneVsRestClassifier(LinearSVC( C = 10.0, class_weight = 'balanced', random_state = 0)).fit(X_train.drop(['文本', '意图', 'keywords','tfidf', 'label', 4912], axis = 1), y_train.values)

In [755]:
pre = clf.predict(X_test.drop(['文本', '意图', 'keywords','tfidf', 'label', 4912], axis = 1))

In [756]:
boolresult = (pre == y_test).value_counts()
print("precision: " + str(boolresult[1] / boolresult.sum()))

precision: 0.7945891783567134


In [757]:
clf

OneVsRestClassifier(estimator=LinearSVC(C=10.0, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=1)